<H1>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;         MAJOR PROJECT</H1>

<H2>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; SENTIMENT ANALYSIS ON IMDB REVIEWS </H2>


You can download the dataset from the Kaggle link below: https://www.kaggle.com/ymanojkumar023/kumarmanoj-bag-of-words-meets-bags-of-popcorn 

Dataset : labeledTrainData.tsv 


In [63]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from bs4 import BeautifulSoup
import re
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords



In [64]:
# We load our dataset here..
df = pd.read_csv('labeledTrainData.tsv',  delimiter="\t", quoting=3)



In [65]:
# Let's look at the head of the dataset:
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [66]:
len(df)

25000

In [67]:
len(df["review"])

25000

In [68]:
# Since stopwords aren't used in NLP we have to clear stopwords. For this purpose we must download stopword wordset from nltk library..
# We do this operation using nltk module..
nltk.download('stopwords')




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## * * * * Data Cleaning Operations * * * *

### First, we will delete HTML tags from review sentences using the BeautifulSoup module.

To explain how these processes are done, first let's choose a single review and see how it is done for you:

In [69]:
sample = df.review[5]
sample

'"I dont know why people think this is such a bad movie. Its got a pretty good plot, some good action, and the change of location for Harry does not hurt either. Sure some of its offensive and gratuitous but this is not the only movie like that. Eastwood is in good form as Dirty Harry, and I liked Pat Hingle in this movie as the small town cop. If you liked DIRTY HARRY, then you should see this one, its a lot better than THE DEAD POOL. 4/5"'

In [70]:
# Clear HTML tags..
sample = BeautifulSoup(sample).get_text()
sample  # After the HTML tags are cleared.

'"I dont know why people think this is such a bad movie. Its got a pretty good plot, some good action, and the change of location for Harry does not hurt either. Sure some of its offensive and gratuitous but this is not the only movie like that. Eastwood is in good form as Dirty Harry, and I liked Pat Hingle in this movie as the small town cop. If you liked DIRTY HARRY, then you should see this one, its a lot better than THE DEAD POOL. 4/5"'

In [71]:
# we clean it from punctuation and numbers - using regex..
sample = re.sub("[^a-zA-Z]",' ',sample)
sample

' I dont know why people think this is such a bad movie  Its got a pretty good plot  some good action  and the change of location for Harry does not hurt either  Sure some of its offensive and gratuitous but this is not the only movie like that  Eastwood is in good form as Dirty Harry  and I liked Pat Hingle in this movie as the small town cop  If you liked DIRTY HARRY  then you should see this one  its a lot better than THE DEAD POOL      '

In [72]:
# we convert it to lowercase, since our machine learning algorithms think capitalized letters with different words
# and this may result mistakes..
sample = sample.lower()
sample

' i dont know why people think this is such a bad movie  its got a pretty good plot  some good action  and the change of location for harry does not hurt either  sure some of its offensive and gratuitous but this is not the only movie like that  eastwood is in good form as dirty harry  and i liked pat hingle in this movie as the small town cop  if you liked dirty harry  then you should see this one  its a lot better than the dead pool      '

In [73]:
# stopwords (stopwords are the words like the, is, are not to be used by AI. These are grammar words..)
# First we split the words with split and convert them to a list. Our goal is to remove stopwords..
sample = sample.split()
  

In [74]:
sample

['i',
 'dont',
 'know',
 'why',
 'people',
 'think',
 'this',
 'is',
 'such',
 'a',
 'bad',
 'movie',
 'its',
 'got',
 'a',
 'pretty',
 'good',
 'plot',
 'some',
 'good',
 'action',
 'and',
 'the',
 'change',
 'of',
 'location',
 'for',
 'harry',
 'does',
 'not',
 'hurt',
 'either',
 'sure',
 'some',
 'of',
 'its',
 'offensive',
 'and',
 'gratuitous',
 'but',
 'this',
 'is',
 'not',
 'the',
 'only',
 'movie',
 'like',
 'that',
 'eastwood',
 'is',
 'in',
 'good',
 'form',
 'as',
 'dirty',
 'harry',
 'and',
 'i',
 'liked',
 'pat',
 'hingle',
 'in',
 'this',
 'movie',
 'as',
 'the',
 'small',
 'town',
 'cop',
 'if',
 'you',
 'liked',
 'dirty',
 'harry',
 'then',
 'you',
 'should',
 'see',
 'this',
 'one',
 'its',
 'a',
 'lot',
 'better',
 'than',
 'the',
 'dead',
 'pool']

In [75]:
len(sample)

88

In [76]:
# sample without stopwords
swords = set(stopwords.words("english"))                      # conversion into set for fast searching
sample = [w for w in sample if w not in swords]               
sample


['dont',
 'know',
 'people',
 'think',
 'bad',
 'movie',
 'got',
 'pretty',
 'good',
 'plot',
 'good',
 'action',
 'change',
 'location',
 'harry',
 'hurt',
 'either',
 'sure',
 'offensive',
 'gratuitous',
 'movie',
 'like',
 'eastwood',
 'good',
 'form',
 'dirty',
 'harry',
 'liked',
 'pat',
 'hingle',
 'movie',
 'small',
 'town',
 'cop',
 'liked',
 'dirty',
 'harry',
 'see',
 'one',
 'lot',
 'better',
 'dead',
 'pool']

In [77]:
len(sample)

43

In [78]:
# After describing the cleanup process, we now batch clean the reviews in our entire dataframe in a loop
# for this purpose we first create a function:

In [79]:
def process(review):
    # without HTML tags
    review = BeautifulSoup(review).get_text()
    # without punctuation and numbers
    review = re.sub("[^a-zA-Z]",' ',review)
    # lowercase and splitting to eliminate stopwords
    review = review.lower()
    review = review.split()
    # without stopwords
    swords = set(stopwords.words("english"))                      
    review = [w for w in review if w not in swords]               
    # we join splitted paragraphs with join before return..
    return(" ".join(review))

In [80]:
# We clean our training data with the help of the above function:
# We can see the status of the review process by printing a line after every 1000 reviews.

train_x_tum = []
for r in range(len(df["review"])):        
    if (r+1)%1000 == 0:        
        print("No of reviews processed =", r+1)
    train_x_tum.append(process(df["review"][r]))

No of reviews processed = 1000
No of reviews processed = 2000
No of reviews processed = 3000
No of reviews processed = 4000
No of reviews processed = 5000
No of reviews processed = 6000
No of reviews processed = 7000
No of reviews processed = 8000
No of reviews processed = 9000
No of reviews processed = 10000
No of reviews processed = 11000
No of reviews processed = 12000
No of reviews processed = 13000
No of reviews processed = 14000
No of reviews processed = 15000
No of reviews processed = 16000
No of reviews processed = 17000
No of reviews processed = 18000
No of reviews processed = 19000
No of reviews processed = 20000
No of reviews processed = 21000
No of reviews processed = 22000
No of reviews processed = 23000
No of reviews processed = 24000
No of reviews processed = 25000


### Train, test split...

In [81]:
# Now we are going to split our data as train and test..
x = train_x_tum
y = np.array(df["sentiment"])

# train test split
train_x, test_x, y_train, y_test = train_test_split(x,y, test_size = 0.1)


### We are building our Bag of Words !

We have cleaned our data, but for the artificial intelligence to work, it is necessary to convert this text-based data into numbers and a matrix called bag of words. Here we use the CountVectorizer tool in sklearn for this purpose:

<IMG src="bag.jpg" width="900" height="900" >

In [82]:
# Using the countvectorizer function in sklearn, we create a bag of words with a maximum of 5000 words...
vectorizer = CountVectorizer( max_features = 5000 )

# We convert our train data to feature vector matrix
train_x = vectorizer.fit_transform(train_x)



In [83]:
train_x

<22500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 1777199 stored elements in Compressed Sparse Row format>

In [84]:
# We are converting it to array because it wants array for fit operation..
train_x = train_x.toarray()
train_y = y_train

In [85]:
train_x.shape, train_y.shape

((22500, 5000), (22500,))

In [86]:
train_y

array([1, 0, 0, ..., 0, 1, 1], dtype=int64)

### We build and fit a Random Forest Model

In [87]:
model = RandomForestClassifier(n_estimators = 100, random_state=42)
model.fit(train_x, train_y)


RandomForestClassifier(random_state=42)

### Now it's time for our test data..

In [88]:
# We convert our test data to feature vector matrix
test_xx = vectorizer.transform(test_x)


In [89]:
test_xx

<2500x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 198060 stored elements in Compressed Sparse Row format>

In [90]:
test_xx = test_xx.toarray()

In [91]:
test_xx.shape

(2500, 5000)

#### Now let's predict..

In [92]:
test_predict = model.predict(test_xx)
acc = roc_auc_score(y_test, test_predict)

In [93]:
print("Accuracy: % ", acc * 100)

Accuracy: %  83.52642873074922
